In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
import numpy as np
from sklearn import metrics

In [42]:
df = pd.read_table("/Users/myazdaniUCSD/Documents/OMI/illumina-16s/data/AG-data/100nt/ag-cleaned_L6.txt")
print df.shape

(9810, 2474)


/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (0,71,116,159,250) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df.iloc[:,466:].head()

,HMP_SITE,k__Archaea;p__Crenarchaeota,k__Archaea;p__Euryarchaeota,k__Archaea;p__[Parvarchaeota],k__Bacteria;p__,k__Bacteria;p__AD3,k__Bacteria;p__Acidobacteria,k__Bacteria;p__Actinobacteria,k__Bacteria;p__AncK6,k__Bacteria;p__Aquificae,...,k__Bacteria;p__WS4;c__;o__;f__;g__,k__Bacteria;p__ZB3;c__;o__;f__;g__,k__Bacteria;p__[Caldithrix];c__KSB1;o__GW-22;f__;g__,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__CM44,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__Deinococcus,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__B-42,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__Truepera,k__Bacteria;p__[Thermi];c__Deinococci;o__Thermales;f__Thermaceae;g__Meiothermus,k__Bacteria;p__[Thermi];c__Deinococci;o__Thermales;f__Thermaceae;g__Thermus
0,FECAL,0.0,0.000000,0.0,0.0,0.0,0.000000,0.001832,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,FECAL,0.0,0.000000,0.0,0.0,0.0,0.000000,0.001634,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,FECAL,0.0,0.000000,0.0,0.0,0.0,0.000000,0.208809,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,SKIN,0.0,0.000000,0.0,0.0,0.0,0.000753,0.630773,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000251,0.0,0.0,0.0,0.0,0.0
4,FECAL,0.0,0.000065,0.0,0.0,0.0,0.000000,0.050539,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [44]:
df_rel = df[df["HMP_SITE"] == "FECAL"]
print df_rel.shape

(8741, 2474)


In [45]:
df_rel.iloc[:,466:].head()

,HMP_SITE,k__Archaea;p__Crenarchaeota,k__Archaea;p__Euryarchaeota,k__Archaea;p__[Parvarchaeota],k__Bacteria;p__,k__Bacteria;p__AD3,k__Bacteria;p__Acidobacteria,k__Bacteria;p__Actinobacteria,k__Bacteria;p__AncK6,k__Bacteria;p__Aquificae,...,k__Bacteria;p__WS4;c__;o__;f__;g__,k__Bacteria;p__ZB3;c__;o__;f__;g__,k__Bacteria;p__[Caldithrix];c__KSB1;o__GW-22;f__;g__,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__CM44,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__Deinococcus,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__B-42,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Trueperaceae;g__Truepera,k__Bacteria;p__[Thermi];c__Deinococci;o__Thermales;f__Thermaceae;g__Meiothermus,k__Bacteria;p__[Thermi];c__Deinococci;o__Thermales;f__Thermaceae;g__Thermus
0,FECAL,0.0,0.000000,0.0,0.0,0.0,0.0,0.001832,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,FECAL,0.0,0.000000,0.0,0.0,0.0,0.0,0.001634,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FECAL,0.0,0.000000,0.0,0.0,0.0,0.0,0.208809,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FECAL,0.0,0.000065,0.0,0.0,0.0,0.0,0.050539,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,FECAL,0.0,0.000000,0.0,0.0,0.0,0.0,0.021245,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# BMI regression

In [46]:
df_rel[["BMI_CORRECTED"]] = df_rel[["BMI_CORRECTED"]].apply(lambda x: pd.to_numeric(x, errors = "coerce"))

In [47]:
df_clean = df_rel.dropna()
print df_clean.shape

(1779, 2474)


In [48]:
df_clean["BMI_CORRECTED"].describe()

count    1779.000000
mean       23.288702
std         5.343387
min         9.630000
25%        20.180000
50%        22.830000
75%        25.830000
max        59.880000
Name: BMI_CORRECTED, dtype: float64

In [49]:
y = np.array(df_clean["BMI_CORRECTED"])
X = np.array(df_clean.iloc[:,467:])
print "y shape", y.shape
print "X shape", X.shape

y shape (1779,)
X shape (1779, 2007)


In [51]:
RF = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [52]:
scores = -1*cross_validation.cross_val_score(RF, X, y, cv=10, scoring = 'mean_absolute_error')

In [53]:
np.mean(scores)

3.4085884482109017

In [54]:
KNN = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=dict(n_neighbors=range(1,11)), n_jobs = -1, scoring = 'mean_absolute_error')
scores = -1*cross_validation.cross_val_score(KNN, X, y, cv=5, scoring = 'mean_absolute_error')
np.mean(scores)

3.9147874707671755

## Age classification

In [ ]:
df_rel[["AGE_CORRECTED"]] = df_rel["AGE_CORRECTED"].apply(lambda x: pd.to_numeric(x, errors = "coerce"))
df_rel["AGE_CORRECTED"].describe()

In [ ]:
df_clean = df_rel.dropna()
print df_clean.shape

In [ ]:
y = np.array(df_clean["AGE_CORRECTED"])
X = np.array(df_clean.iloc[:,467:])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
RF = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
RF.fit(X_train, y_train)

In [ ]:
RF.score(X_train, y_train)

In [ ]:
RF.score(X_test, y_test)

In [ ]:
scores = -1*cross_validation.cross_val_score(RF, X, y, cv=5, scoring = 'mean_absolute_error')

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
KNN = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=dict(n_neighbors=range(10,20)), n_jobs = -1, scoring = 'mean_absolute_error')
scores = -1*cross_validation.cross_val_score(KNN, X, y, cv=10, scoring = 'mean_absolute_error')
np.mean(scores)